In [1]:
import sys
import json
import re
from collections import defaultdict
from typing import List, Dict
import torch
import subprocess
from sklearn.model_selection import train_test_split
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

## Number of few shot examples = 3 / 5 /10

In [2]:
FINE_LABELS = [
        "Ad-hominem", "Appeal-to-authority", "Appeal-to-emotion",
        "Causal-oversimplification", "Cherry-picking", "Circular-reasoning",
        "Doubt", "Evading-the-burden-of-proof", "False-analogy",
        "False-dilemma", "Flag-waving", "Hasty-generalization",
        "Loaded-language", "Name-calling-or-labelling", "Red-herring",
        "Slippery-slope", "Slogan", "Strawman",
        "Thought-terminating-cliches", "Vagueness"
    ]

In [3]:
Model_Name = "Qwen/Qwen2.5-3B-Instruct"
# Model_Name = "Qwen/Qwen2.5-7B-Instruct"
# Model_Name = "mistralai/Mistral-7B-Instruct-v0.3"
# Model_Name = "google/gemma-3n-E2B-it"
# Model_Name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# Model_Name = "microsoft/Phi-3-mini-4k-instruct"
# Model_Name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

In [4]:
def init_model(model_name=Model_Name, use_cpu=False, use_4bit=False):
    device = "cuda" if torch.cuda.is_available() and not use_cpu else "cpu"
    print(f"Loading model on device: {device}")

    if use_4bit:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            low_cpu_mem_usage=True,
            trust_remote_code=True
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            trust_remote_code=True
        )

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer, device

In [5]:
def parse_conll_file(filepath: str) -> List[Dict]:
    posts = []
    current_post = None
    
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.rstrip('\n')
            
            if line.startswith("# post_id = "):
                if current_post:
                    posts.append(current_post)
                current_post = {
                    'post_id': line.split(" = ")[1],
                    'date': '',
                    'topic': '',
                    'text': '',
                    'tokens': [],
                    'bio_tags_a1': [],
                    'bio_tags_a2': []
                }
            elif line.startswith("# post_date = "):
                current_post['date'] = line.split(" = ")[1]
            elif line.startswith("# post_topic_keywords = "):
                current_post['topic'] = line.split(" = ")[1]
            elif line.startswith("# post_text = "):
                current_post['text'] = line.split(" = ")[1]
            elif len(line.strip()) == 0:
                if current_post and current_post['tokens']:
                    current_post['spans_a1'] = bio_to_spans(
                        current_post['tokens'], current_post['bio_tags_a1']
                    )
                    current_post['spans_a2'] = bio_to_spans(
                        current_post['tokens'], current_post['bio_tags_a2']
                    )
                    posts.append(current_post)
                    current_post = None
            elif current_post is not None:
                parts = line.split('\t')
                if len(parts) >= 4:
                    current_post['tokens'].append(parts[1])
                    current_post['bio_tags_a1'].append(parts[2])
                    current_post['bio_tags_a2'].append(parts[3])
    
    if current_post and current_post['tokens']:
        current_post['spans_a1'] = bio_to_spans(
            current_post['tokens'], current_post['bio_tags_a1']
        )
        current_post['spans_a2'] = bio_to_spans(
            current_post['tokens'], current_post['bio_tags_a2']
        )
        posts.append(current_post)
    
    print(f"Parsed {len(posts)} posts from {filepath}")
    return posts

In [6]:
def bio_to_spans(tokens: List[str], bio_tags: List[str]) -> List[Dict]:
    active_spans = {}
    completed_spans = []
    
    for idx, (token, bio_tag_str) in enumerate(zip(tokens, bio_tags)):
        bio_tags_list = bio_tag_str.split('|')
        current_labels = set()
        
        for bio_tag in bio_tags_list:
            bio_tag = bio_tag.strip()
            if bio_tag.startswith("B-"):
                label = bio_tag[2:]
                current_labels.add(label)
                
                if label in active_spans:
                    span = active_spans[label]
                    completed_spans.append({
                        'label': label,
                        'start_tok': span['start_tok'],
                        'end_tok': idx - 1,
                        'text': " ".join(span['tokens'])
                    })
                
                active_spans[label] = {'start_tok': idx, 'tokens': [token]}
                
            elif bio_tag.startswith("I-"):
                label = bio_tag[2:]
                current_labels.add(label)
                if label in active_spans:
                    active_spans[label]['tokens'].append(token)
                else:
                    active_spans[label] = {'start_tok': idx, 'tokens': [token]}
        
        labels_to_close = set(active_spans.keys()) - current_labels
        for label in labels_to_close:
            span = active_spans[label]
            completed_spans.append({
                'label': label,
                'start_tok': span['start_tok'],
                'end_tok': idx - 1,
                'text': " ".join(span['tokens'])
            })
            del active_spans[label]
    
    for label, span in active_spans.items():
        completed_spans.append({
            'label': label,
            'start_tok': span['start_tok'],
            'end_tok': len(tokens) - 1,
            'text': " ".join(span['tokens'])
        })
    
    return completed_spans

In [7]:
def format_for_model(prompt: str, tokenizer) -> str:
    messages = [{"role": "user", "content": prompt}]
    if hasattr(tokenizer, "apply_chat_template"):
        try:
            return tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        except Exception:
            return prompt
    return prompt

In [8]:
def generate(prompt: str, model, tokenizer, device, max_tokens: int = 1024, temperature: float = 0.1) -> str:
    text = format_for_model(prompt, tokenizer)
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True if temperature > 0 else False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
    
    generated = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)
    
    del inputs, outputs, generated
    torch.cuda.empty_cache()
    
    return response

In [9]:
def extract_spans_from_response(response: str) -> List[Dict]:
    spans = []
    response = re.sub(r'```json\s*', '', response)
    response = re.sub(r'```\s*', '', response)
    response = response.strip()
    
    # Strategy 1: Try JSON array
    try:
        # Find JSON array
        start = response.find("[")
        end = response.rfind("]") + 1
        if start != -1 and end > start:
            json_str = response[start:end]
            data = json.loads(json_str)
            if isinstance(data, list):
                spans = data
                print(f" Extracted from JSON array")
    except Exception as e:
        print(f" JSON array parsing failed: {e}")
    
    # Strategy 2: Try JSON object with "spans" key
    if not spans:
        try:
            start = response.find("{")
            end = response.rfind("}") + 1
            if start != -1 and end > start:
                json_str = response[start:end]
                data = json.loads(json_str)
                if "spans" in data:
                    spans = data["spans"]
                    print(f" Extracted from JSON object")
        except Exception as e:
            print(f" JSON object parsing failed: {e}")
    
    # Strategy 3: Line-by-line extraction (for malformed JSON)
    if not spans:
        print(f" Trying line-by-line extraction...")
        for line in response.split('\n'):
            text_match = re.search(r'"text"\s*:\s*"([^"]+)"', line)
            label_match = re.search(r'"label"\s*:\s*"([^"]+)"', line)
            if text_match and label_match:
                spans.append({
                    "text": text_match.group(1),
                    "label": label_match.group(1)
                })
        if spans:
            print(f" Extracted {len(spans)} spans line-by-line")
    
    # Validate spans
    valid_spans = []
    for span in spans:
        if not isinstance(span, dict):
            continue
        
        text = span.get("text", "").strip()
        label = span.get("label", "").strip().replace(" ", "-").replace("_", "-")
        
        if not text or not label:
            continue
        
        # Match label
        if label in FINE_LABELS:
            valid_spans.append({"text": text, "label": label})
        else:
            for fine_label in FINE_LABELS:
                if fine_label.lower() == label.lower():
                    valid_spans.append({"text": text, "label": fine_label})
                    break
    
    return valid_spans

In [10]:
def build_few_shot_examples(posts: List[Dict], n_examples_per_label: int = 3, 
                           annotator: str = "a1") -> Dict[str, List[str]]:
    examples = defaultdict(list)
    spans_key = f'spans_{annotator}'
    
    # Collect all spans per label
    label_spans = defaultdict(list)
    for post in posts:
        for span in post.get(spans_key, []):
            label = span['label']
            text = span['text'].strip()
            if label in FINE_LABELS and len(text.split()) >= 2:  # At least 2 words
                label_spans[label].append(text)
    
    # Select diverse examples
    for label, span_list in label_spans.items():
        # Remove duplicates
        unique_spans = list(set(span_list))
        
        # Sort by length to get variety
        sorted_spans = sorted(unique_spans, key=len)
        
        # Pick examples from different length buckets
        selected = []
        if len(sorted_spans) <= n_examples_per_label:
            selected = sorted_spans
        else:
            step = len(sorted_spans) // n_examples_per_label
            for i in range(n_examples_per_label):
                idx = min(i * step, len(sorted_spans) - 1)
                selected.append(sorted_spans[idx])
        
        examples[label] = selected
    
    print(f"\nBuilt few-shot examples for {len(examples)} labels:")
    for label in FINE_LABELS:
        if label in examples:
            print(f"  {label}: {len(examples[label])} examples")
    
    return dict(examples)

In [11]:
def create_few_shot_prompt(text: str, examples: Dict[str, List[str]], 
                          max_examples_per_label: int = 3) -> str:    
    prompt = """You are an expert in identifying logical fallacies in text. 
Your task is to extract ALL text spans that contain fallacies and label them.

AVAILABLE FALLACY TYPES AND EXAMPLES:
"""
    
    # Add examples for each label
    for label in FINE_LABELS:
        if label in examples and examples[label]:
            prompt += f"\n{label}:\n"
            for ex in examples[label][:max_examples_per_label]:
                prompt += f'  - "{ex}"\n'
    
    prompt += f"""
Now extract ALL fallacy spans from this text:

TEXT: "{text}"

INSTRUCTIONS:
1. Find ALL spans that contain fallacies (don't miss any!)
2. A span can be a phrase, sentence, or multiple sentences
3. Multiple spans can have the same label
4. Return valid JSON with this format:
{{"spans": [{{"text": "exact text from input", "label": "Label-name"}}]}}
If no fallacies found: {{"spans": []}}
Your response (JSON only):"""    
    return prompt

In [12]:
def predict_spans(text: str, examples: Dict[str, List[str]], model, tokenizer, device) -> List[Dict]:
    prompt = create_few_shot_prompt(text, examples, max_examples_per_label=3)
    response = generate(prompt, model, tokenizer, device, max_tokens=1024, temperature=0.1)
    
    print(f"\n  MODEL RESPONSE (first 500 chars):")
    print(f"  {response[:500]}")
    print(f"  ...")
    
    spans = extract_spans_from_response(response)
    print(f"\n  Extracted {len(spans)} spans:")
    for span in spans[:5]:  # Show first 5
        print(f"    - [{span['label']}] \"{span['text'][:60]}...\"")
    
    return spans

In [13]:
def spans_to_bio_improved(text: str, tokens: List[str], spans: List[Dict]) -> List[str]:
    token_labels = [set() for _ in tokens]
    
    # Build reconstructed text
    reconstructed = " ".join(tokens)
    reconstructed_lower = reconstructed.lower()
    
    print(f"\n  Converting {len(spans)} spans to BIO tags...")
    
    for span in spans:
        span_text = span['text'].strip()
        label = span['label']
        matched = False
        
        # Strategy 1: Case-insensitive substring match in reconstructed text
        span_text_lower = span_text.lower()
        span_start = reconstructed_lower.find(span_text_lower)
        
        if span_start != -1:
            # Count tokens before this position
            prefix = reconstructed[:span_start]
            tokens_before = len(prefix.split()) if prefix.strip() else 0
            
            # Count tokens in span
            span_token_count = len(span_text.split())
            
            start_idx = tokens_before
            end_idx = min(start_idx + span_token_count, len(tokens))
            
            if start_idx < len(tokens):
                token_labels[start_idx].add(f"B-{label}")
                for idx in range(start_idx + 1, end_idx):
                    token_labels[idx].add(f"I-{label}")
                matched = True
                print(f" Matched '{span_text[:40]}...' -> tokens[{start_idx}:{end_idx}]")
                continue
        
        # Strategy 2: Token-level fuzzy matching 
        if not matched:
            span_tokens = span_text_lower.split()
            if not span_tokens:
                continue
            
            tokens_lower = [t.lower() for t in tokens]
            best_match_start = -1
            best_match_score = 0.0
            
            # Sliding window
            for i in range(len(tokens_lower) - len(span_tokens) + 1):
                window = tokens_lower[i:i + len(span_tokens)]
                
                # Calculate match score
                matches = sum(1 for w, s in zip(window, span_tokens) if w == s or w in s or s in w)
                score = matches / len(span_tokens)
                
                if score > best_match_score:
                    best_match_score = score
                    best_match_start = i
            
            # Apply if reasonable match 
            if best_match_start >= 0 and best_match_score >= 0.5:
                end_idx = best_match_start + len(span_tokens)
                token_labels[best_match_start].add(f"B-{label}")
                for idx in range(best_match_start + 1, end_idx):
                    token_labels[idx].add(f"I-{label}")
                matched = True
                print(f" Fuzzy matched ({best_match_score:.2f}) '{span_text[:40]}...' -> tokens[{best_match_start}:{end_idx}]")
        
        # Strategy 3: Check if span tokens appear anywhere in sequence
        if not matched:
            span_tokens = span_text_lower.split()
            if len(span_tokens) >= 3:  # Only for longer spans
                # Try to find first and last token
                tokens_lower = [t.lower() for t in tokens]
                first_token = span_tokens[0]
                last_token = span_tokens[-1]
                
                # Find all occurrences of first token
                for i, tok in enumerate(tokens_lower):
                    if first_token in tok or tok in first_token:
                        # Check if last token appears within reasonable distance
                        for j in range(i + 1, min(i + len(span_tokens) + 3, len(tokens_lower))):
                            if last_token in tokens_lower[j] or tokens_lower[j] in last_token:
                                # Found a match!
                                token_labels[i].add(f"B-{label}")
                                for idx in range(i + 1, j + 1):
                                    token_labels[idx].add(f"I-{label}")
                                matched = True
                                print(f" Loose matched '{span_text[:40]}...' -> tokens[{i}:{j+1}]")
                                break
                    if matched:
                        break
        
        if not matched:
            print(f" FAILED to match: '{span_text[:50]}...'")
    
    # Convert to BIO format with deduplication
    bio_tags = []
    for label_set in token_labels:
        if not label_set:
            bio_tags.append("O")
        else:
            # Deduplicate: if both B- and I- exist for same label, keep only B-
            deduplicated = set()
            labels_seen = set()
            
            for tag in sorted(label_set):
                if tag.startswith("B-"):
                    label = tag[2:]
                    labels_seen.add(label)
                    deduplicated.add(tag)
                elif tag.startswith("I-"):
                    label = tag[2:]
                    # Only add I- if we haven't seen B- for this label
                    if label not in labels_seen:
                        deduplicated.add(tag)
            
            bio_tags.append("|".join(sorted(deduplicated)))
    
    # FIX: Clean up any I- tags that don't have a preceding B-
    bio_tags = fix_bio_sequence(bio_tags)
    
    # VALIDATION: Check for malformed BIO sequences
    validate_bio_tags(bio_tags, tokens)
    
    return bio_tags

In [14]:
def fix_bio_sequence(bio_tags: List[str]) -> List[str]:
    fixed_tags = []
    
    for idx, tag in enumerate(bio_tags):
        if tag == "O":
            fixed_tags.append(tag)
            continue
        
        # Process each label in multi-label tags
        labels_with_b = set()  # Labels that have B- in this token
        labels_with_i = set()  # Labels that have I- in this token
        
        for single_tag in tag.split("|"):
            if single_tag.startswith("B-"):
                label = single_tag[2:]
                labels_with_b.add(label)
            elif single_tag.startswith("I-"):
                label = single_tag[2:]
                labels_with_i.add(label)
        
        # Build the fixed tag list
        final_labels = []
        
        # Add all B- tags (they're always valid)
        for label in sorted(labels_with_b):
            final_labels.append(f"B-{label}")
        
        # Process I- tags (only if no B- for same label and previous token has it)
        for label in sorted(labels_with_i):
            # Skip if we already have B- for this label
            if label in labels_with_b:
                continue
            
            # Check if previous token has this label
            if idx > 0:
                prev_tag = fixed_tags[idx - 1]
                if f"B-{label}" in prev_tag or f"I-{label}" in prev_tag:
                    # Valid continuation
                    final_labels.append(f"I-{label}")
                else:
                    # Orphan I- becomes B-
                    final_labels.append(f"B-{label}")
            else:
                # First token can't have I-, convert to B-
                final_labels.append(f"B-{label}")
        
        if final_labels:
            fixed_tags.append("|".join(sorted(final_labels)))
        else:
            fixed_tags.append("O")
    
    return fixed_tags

In [15]:
def validate_bio_tags(bio_tags: List[str], tokens: List[str]):
    issues = []
    active_spans = {}
    span_count = 0
    
    for idx, tag in enumerate(bio_tags):
        if tag == "O":
            for label in list(active_spans.keys()):
                span = active_spans[label]
                expected_end = idx - 1
                if span['start'] > expected_end:
                    issues.append(f"Invalid span for {label}: start={span['start']} > end={expected_end}")
                span_count += 1
                del active_spans[label]
            continue
        current_labels = set()
        for single_tag in tag.split("|"):
            if single_tag.startswith("B-"):
                label = single_tag[2:]
                current_labels.add(label)
                
                # Close existing span if any
                if label in active_spans:
                    span = active_spans[label]
                    expected_end = idx - 1
                    if span['start'] > expected_end:
                        issues.append(f"Invalid span for {label}: start={span['start']} > end={expected_end}")
                    span_count += 1
                
                # Start new span
                active_spans[label] = {'start': idx, 'tokens': [tokens[idx]]}
                
            elif single_tag.startswith("I-"):
                label = single_tag[2:]
                current_labels.add(label)
                
                if label not in active_spans:
                    issues.append(f"I-{label} at position {idx} without preceding B-")
                else:
                    active_spans[label]['tokens'].append(tokens[idx])
        
        # Close spans not in current labels
        for label in list(active_spans.keys()):
            if label not in current_labels:
                span = active_spans[label]
                expected_end = idx - 1
                if span['start'] > expected_end:
                    issues.append(f"Invalid span for {label}: start={span['start']} > end={expected_end}")
                span_count += 1
                del active_spans[label]
    
    # Close remaining spans
    for label in active_spans:
        span_count += 1
    
    if issues:
        print(f"    WARNING: BIO validation found {len(issues)} issues:")
        for issue in issues[:5]:
            print(f"    - {issue}")
    else:
        print(f"   BIO validation passed: {span_count} spans")
    
    return bio_tags

In [16]:
def write_conll_predictions(posts: List[Dict], output_path: str, 
                           examples: Dict[str, List[str]], model, tokenizer, device):
    with open(output_path, 'w', encoding='utf-8') as f:
        for idx, post in enumerate(posts):
            print(f"\nProcessing post {idx+1}/{len(posts)}: {post['post_id']}")
            
            # Write metadata
            f.write(f"# post_id = {post['post_id']}\n")
            f.write(f"# post_date = {post['date']}\n")
            f.write(f"# post_topic_keywords = {post['topic']}\n")
            f.write(f"# post_text = {post['text']}\n")
            
            # Predict spans
            predicted_spans = predict_spans(post['text'], examples, model, tokenizer, device)
            
            # Convert to BIO tags
            bio_tags = spans_to_bio_improved(post['text'], post['tokens'], predicted_spans)
            
            # Ensure we have the right number of tokens
            if len(bio_tags) != len(post['tokens']):
                print(f"   WARNING: Token count mismatch! tokens={len(post['tokens'])}, bio_tags={len(bio_tags)}")
                # Pad or truncate
                if len(bio_tags) < len(post['tokens']):
                    bio_tags.extend(['O'] * (len(post['tokens']) - len(bio_tags)))
                else:
                    bio_tags = bio_tags[:len(post['tokens'])]
            for tok_idx, (token, bio_tag) in enumerate(zip(post['tokens'], bio_tags), start=1):
                f.write(f"{tok_idx}\t{token}\t{bio_tag}\t{bio_tag}\n")
            f.write("\n")
    
    print(f"\nPredictions saved to: {output_path}")

In [17]:
def evaluate(gold_path: str, pred_path: str):    
    cmd = [
        sys.executable,
        "eval.py",
        "-S", "B",
        "-G", gold_path,
        "-P", pred_path
    ]
    
    print("\n" + "="*60)
    print("RUNNING EVALUATION - SUBTASK B")
    print("="*60)
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)

In [19]:
if __name__ == "__main__":
    CONLL_PATH = "../data/faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll"
    safe_model_name = Model_Name.replace("/", "_")
    PRED_PATH = f"../results/subtask-b/{safe_model_name}_predicted.tsv"
    GOLD_PATH = f"../results/subtask-b/{safe_model_name}_gold.tsv"
    
    
    print("Initializing span extractor...")
    model, tokenizer, device = init_model(use_4bit=True)
    
    print("\nParsing CoNLL file...")
    all_posts = parse_conll_file(CONLL_PATH)
    
    train_posts, test_posts = train_test_split(
        all_posts, test_size=0.2, random_state=42, shuffle=True
    )
    print(f"\nTrain: {len(train_posts)} posts")
    print(f"Test: {len(test_posts)} posts")
    
    print("\nBuilding few-shot examples...")
    examples = build_few_shot_examples(train_posts, n_examples_per_label=3, annotator="a1")
    
    print("\nGenerating predictions...")
    write_conll_predictions(test_posts, PRED_PATH, examples, model, tokenizer, device)
    
    print("\nWriting gold labels...")
    with open(GOLD_PATH, 'w', encoding='utf-8') as f:
        for post in test_posts:
            f.write(f"# post_id = {post['post_id']}\n")
            f.write(f"# post_date = {post['date']}\n")
            f.write(f"# post_topic_keywords = {post['topic']}\n")
            f.write(f"# post_text = {post['text']}\n")
            for tok_idx, (token, bio_a1, bio_a2) in enumerate(zip(
                post['tokens'], post['bio_tags_a1'], post['bio_tags_a2']
            ), start=1):
                f.write(f"{tok_idx}\t{token}\t{bio_a1}\t{bio_a2}\n")
            f.write("\n")
    
    print(f"Gold labels saved to: {GOLD_PATH}")    
    evaluate(GOLD_PATH, PRED_PATH)

Initializing span extractor...
Loading model on device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Parsing CoNLL file...
Parsed 1152 posts from ../data/faina-v1.0_fadeit-shared-task/faina-v1.0_fadeit-shared-task/data/subtask-b/train-dev.conll

Train: 921 posts
Test: 231 posts

Building few-shot examples...

Built few-shot examples for 20 labels:
  Ad-hominem: 3 examples
  Appeal-to-authority: 3 examples
  Appeal-to-emotion: 3 examples
  Causal-oversimplification: 3 examples
  Cherry-picking: 3 examples
  Circular-reasoning: 3 examples
  Doubt: 3 examples
  Evading-the-burden-of-proof: 3 examples
  False-analogy: 3 examples
  False-dilemma: 3 examples
  Flag-waving: 3 examples
  Hasty-generalization: 3 examples
  Loaded-language: 3 examples
  Name-calling-or-labelling: 3 examples
  Red-herring: 3 examples
  Slippery-slope: 3 examples
  Slogan: 3 examples
  Strawman: 3 examples
  Thought-terminating-cliches: 3 examples
  Vagueness: 3 examples

Generating predictions...

Processing post 1/231: 353

  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 E


  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 19/231: 379

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Se queste cose non si fanno è una scelta dei parlamentari, prescindendo dal governo", "label": "Flag-waving"}]}
  ...
 Extracted from JSON array

  Extracted 1 spans:
    - [Flag-waving] "Se queste cose non si fanno è una scelta dei parlamentari, p..."

  Converting 1 spans to BIO tags...
 Fuzzy matched (0.79) 'Se queste cose non si fanno è una scelta...' -> tokens[29:43]
   BIO validation passed: 1 spans

Processing post 20/231: 1424

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Francamente sentire qualcuno sostenere che i cadaveri trovati ad Auschwitz furono dovuti a una epidemia di tifo  e che l'Olocausto non è mai esistito, fa rabbrividire,


  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Questo è il Modello #Milano caro al Sindaco [USER]? Evidentemente la Giunta adesso si focalizza solo sui rincari #Atm e il resto può aspettare...", "label": "Red-herring"}]}
  ...
 Extracted from JSON array

  Extracted 1 spans:
    - [Red-herring] "Questo è il Modello #Milano caro al Sindaco [USER]? Evidente..."

  Converting 1 spans to BIO tags...
 Loose matched 'Questo è il Modello #Milano caro al Sind...' -> tokens[18:41]
   BIO validation passed: 1 spans

Processing post 40/231: 808

  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 41/231: 373

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Poi se parliamo di propaganda vs realtà, allora sì, il problema è enorme  #GovernoDraghi", "label": "Vagueness"}]

...


  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 141/231: 1298

  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 142/231: 1347

  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 143/231: 928

  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:


  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 163/231: 733

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "se Salvini non fa sbarcare gli immigrati per 4 g, per il controllo dei documenti, è sequestro di persona 2) se la Lamorgese non li fa sbarcare per 11 g, perché ci sono le elecciones, nessuno dice niente. E ancora crediamo che la legge sia uguale per tutti! Che paese di merda!", "label": "False-dilemma"}}
  ...
 JSON object parsing failed: Expecting ',' delimiter: line 1 column 326 (char 325)
  → Trying line-by-line extraction...
 Extracted 1 spans line-by-line

  Extracted 1 spans:
    - [False-dilemma] "se Salvini non fa sbarcare gli immigrati per 4 g, per il con..."

  Converting 1 spans to BIO tags...
 Loose matched 'se Salvini non fa sbarcare gli immigr


  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 184/231: 1086

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Se #Renzi salisse sulla #SeaWatch , sono sicuro che parte della sinistra si schiererebbe con #Salvini per affondare la nave con tutti i migranti. #Scendeteli", "label": "Slippery-slope"}}
  ...
 JSON object parsing failed: Expecting ',' delimiter: line 1 column 208 (char 207)
  → Trying line-by-line extraction...
 Extracted 1 spans line-by-line

  Extracted 1 spans:
    - [Slippery-slope] "Se #Renzi salisse sulla #SeaWatch , sono sicuro che parte de..."

  Converting 1 spans to BIO tags...
 Fuzzy matched (0.56) 'Se #Renzi salisse sulla #SeaWatch , sono...' -> tokens[2:27]
   BIO validation passed: 1 spans

Processing post 185/231: 1182

  MODEL RESPONSE (f


  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 203/231: 783

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Quindi?", "label": "Slogan"}, {"text": "In certe parti del mondo sono normali pure lapidazione, legge del taglione, spose bambine.", "label": "Slippery-slope"}]}
  ...
 Extracted from JSON array

  Extracted 2 spans:
    - [Slogan] "Quindi?..."
    - [Slippery-slope] "In certe parti del mondo sono normali pure lapidazione, legg..."

  Converting 2 spans to BIO tags...
 Fuzzy matched (1.00) 'Quindi?...' -> tokens[36:37]
 Fuzzy matched (0.64) 'In certe parti del mondo sono normali pu...' -> tokens[36:50]
   BIO validation passed: 2 spans

Processing post 204/231: 1015

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "Si può essere più infami?", "lab


  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 224/231: 1350

  MODEL RESPONSE (first 500 chars):
  {"spans": []}
  ...
 Extracted from JSON array
 Extracted from JSON object
  → Trying line-by-line extraction...

  Extracted 0 spans:

  Converting 0 spans to BIO tags...
   BIO validation passed: 0 spans

Processing post 225/231: 474

  MODEL RESPONSE (first 500 chars):
  {"spans": [{"text": "rifugiatevi nelle chiese!", "label": "Thought-terminating-cliches"}]}
  ...
 Extracted from JSON array

  Extracted 1 spans:
    - [Thought-terminating-cliches] "rifugiatevi nelle chiese!..."

  Converting 1 spans to BIO tags...
 Fuzzy matched (1.00) 'rifugiatevi nelle chiese!...' -> tokens[6:9]
   BIO validation passed: 1 spans

Processing post 226/231: 845

  MODEL RESPONSE (first 500